## Problem 4 ##

NOTE TO GRADER: I referenced the starter code as instructed to do this problem. I also used our lecture notes for HMM.

## Hidden Markov Models (HMM)

HMM are popular approaches to working with time series problems where there are latent or hidden states

There are 4 common inference problems in HMM:

- __Filtering__  Inferring the present - carried out by passing messages up and to the right, so infer $h_t$ from $p(h_t|v_{1:t})$  where  $t = T$

- __Prediction__ Inferring the future - similar to filtering but with a new future state, so infer $h_t$ from $p(h_t|v_{1:T})$ where $t>T$

- __Smoothing__  Inferring the past - combine filtering messages with messages up and to the left, so infer $h_t$ from $p(h_t|v_{1:T})$ where $t <T$

- __Decoding__ Find the most likely hidden path - computed similarly to smoothing, so infer the most likely hidden sequence $h_{1:T}$ from $p(h_{1:T}|v_{1:T})$

In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 12 20:35:27 2020

@author: donaldbrown
"""

import numpy as np
import pandas as pd

class HMM:
    """Creates a class for Hidden Markov Models
    Input:
        Viz:      List of observed or visible states over time
        Trans_M:  Numpy array of the Transition matrix for hidden states, 
                  H X H, H=len(Trans_M), no. of hidden variables
        Obs_M:    Numpy array of the Observation matrix, 
                  H X V, V = no. of visible variables
        Pi:       List of initial state probabilities
    Methods:
        filter = The posterior probabilities for hidden states for each time period, T X H array
        smoother = The probabiliteis for the hidden states at each prevoius time period, T X H array
        viturbi = The most likely path of hidden states given the observed state, data frame, 1 X T
        predictor = The probabilities for next hidden state and the next observed state, 1 X H array """
    
    def __init__(self,Viz, Trans_M, Obs_M, Pi):
        # initialize variables
        # Hidden state transition matrix
        self.Trans_M = Trans_M
        # Visible or observates state probabilities given the hidden states
        self.Obs_M = Obs_M
        # No. of hidden states
        self.H = Trans_M.shape[0]
        # No. of observed states
        self.V = Obs_M.shape[0]
        # prior probabaiities for the hidden states
        self.Pi = Pi
        # List of observed states over time
        self.Viz = Viz

        
    def filter(self):
        
        T = len(self.Viz)
        
        # Obtain the joint probabilities of the hidden and observed states at time t
        self.alpha = np.zeros((T, self.H))
        self.alpha[0, :] = self.Pi * self.Obs_M[:,self.Viz[0]]
 
        for t in range(1, T):
            for j in range(self.H):
                self.alpha[t, j] = self.alpha[t - 1].dot(self.Trans_M[:, j]) * self.Obs_M[j, self.Viz[t]]
        
        ### Insert your code here to compute the posterior probabilities ###
        self.Post=np.zeros((T,self.H))
        for t in range(0,T):
            self.Post[t]=self.alpha[t]/np.sum(self.alpha[t])
        #self.Post[0]=self.alpha[0]/np.sum(self.alpha[0])
        #self.Post = self.alpha / (np.sum(self.alpha))
        
        print("Posterior")
        print(self.Post)  
        return self.Post
      
    def smoother(self):

        T = len(self.Viz)
        beta = np.zeros((T, self.H))
 
        # setting beta(T) = 1
        beta[T - 1] = np.ones((self.H))
 
        # Loop backwards way from T-1 to 1
        # Due to python indexing the actual loop will be T-2 to 0
        for t in range(T- 2, -1, -1):
            for j in range(self.H):
                beta[t, j] = (beta[t + 1] * self.Obs_M[:, self.Viz[t + 1]]).dot(self.Trans_M[j, :])
                
        # Obtain the posterior probabilities of the hidden states given the observed states       
        
        ### Insert your code here to compute the posterior probabilities ###
        
        ## Get the alpha
        alpha = np.zeros((T, self.H))
        alpha[0, :] = self.Pi * self.Obs_M[:,self.Viz[0]]
 
        for t in range(1, T):
            for j in range(self.H):
                alpha[t, j] = alpha[t - 1].dot(self.Trans_M[:, j]) * self.Obs_M[j, self.Viz[t]]
        
        ## Get the posterior
        Post=np.zeros((T,self.H))
        for t in range(0,T):
            Post[t]=(alpha[t]*beta[t])/np.sum(alpha[t]*beta[t])
        
        print("beta")
        print(beta)
        print("Posterior")
        print(Post)
        print("alpha")
        print(alpha)
 
        return Post
    
    
    def viturbi(self):
        T = len(self.Viz)
        
        # Obtain the joint probabity of the most likely path that ends in state j at time t
        delta = np.zeros((T, self.H))
        delta[0, :] = (self.Pi * self.Obs_M[:, Viz[0]])
 

        prev = np.zeros((T, self.H))
        prev[0,:] = np.repeat(None, 3)
 
        for t in range(1, T):
            for j in range(self.H):
                # The most likely state given our previoius state at t-1
                
                prob = delta[t - 1]* (self.Trans_M[:, j])
 
                #  The probability of the most probable state given the previous state and the observation at time t
                
                delta[t, j] = np.max(prob) * (self.Obs_M[j, Viz[t]])                
                
                # The most probable state given previous state 

                prev[t, j] = np.argmax(prob)
 
                
        # Path Array
        S = np.zeros(T)
 
        # Find the most probable last hidden state
        last_state = np.argmax(delta[T-1, :])
 
        S[T-1] = last_state
        
        # Find the most probable hidden states at the previous times
        ### Insert your code here ###
            
        # Change to states numbers in problem (i.e., +1)
        S = S+1
            
        S = S.reshape([1,3])
 
        # Path, S, as a dataframe 
        # Create a list of column names, Time  
        cols = list()
        for i in range(1,T+1):
            cols.append("Time "+(str(i)))
        Path = pd.DataFrame(S, columns = cols)
        print('delta')
        print(delta)
        print('Previous')
        print(prev)        
        print("Path")
        print(Path)
        return Path
 

    def predictor(self, steps = 1):
        T = len(self.Viz)
        # Hidden state prediction probabilities using filtering results (Post)
        
        ### Insert your code here ###
        
        temp=self.filter()
        
        for step in range(steps):
            for room in T:
                new_hmm=HMM(self.Viz, self.Trans_M, self.Obs_M, temp[step])
                temp=new_hmm.filter()
                temp=temp[step]
        
        Pred_Hidden = temp
        
        print("Predicted Hidden State")
        print(Pred_Hidden)
        # Visible state prediction using the predicted hidden state probabilities
        ### Insert your code here ###
        
        p=Pred_Hidden
        


        Pred_Visible = p
        print("Predicted Visible State")
        print(Pred_Visible)

In [30]:
## Play cell

test_alpha=np.array([.6,0,0])
np.sum(test_alpha)
test_alpha/np.sum(test_alpha)

array([1., 0., 0.])

In [31]:
# Example lecture problem: Burglar in an apartment
# Data 
# Transition matrix
TM = np.array([[.1,.4,.5],[.4,.0,.6],[0,.6,.4]])  
# Observation matrix
OM = np.array([[.6,.2,.2],[.2,.6,.2],[.2,.2,.6]])
OM = OM.T
# Prior probabilities of hidden states
p = [1,0,0]
# Observed visible states
Viz = [0,2,2]


In [23]:
# Filtering results

hmm1 = HMM(Viz, TM, OM, p)
hmm1.viturbi()

delta
[[0.6     0.      0.     ]
 [0.012   0.048   0.18   ]
 [0.00384 0.0216  0.0432 ]]
Previous
[[nan nan nan]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]
Path
   Time 1  Time 2  Time 3
0     1.0     1.0     3.0


,Time 1,Time 2,Time 3
0,1.0,1.0,3.0


## Burglary Problem from the Lecture

In [2]:
# Example lecture problem: Burglar in an apartment
# Data 
# Transition matrix
TM = np.array([[.1,.4,.5],[.4,.0,.6],[0,.6,.4]])  
# Observation matrix
OM = np.array([[.6,.2,.2],[.2,.6,.2],[.2,.2,.6]])
OM = OM.T
# Prior probabilities of hidden states
p = [1,0,0]
# Observed visible states
Viz = [0,2,2]


In [3]:
# Filtering results

hmm1 = HMM(Viz, TM, OM, p)
hmm1.filter()



self.alpha
[[0.6     0.      0.     ]
 [0.012   0.048   0.18   ]
 [0.00408 0.02256 0.06408]]
Posterior
[[1.         0.         0.        ]
 [0.05       0.2        0.75      ]
 [0.04497354 0.24867725 0.70634921]]


array([[1.        , 0.        , 0.        ],
       [0.05      , 0.2       , 0.75      ],
       [0.04497354, 0.24867725, 0.70634921]])

In [4]:
# Smoothing results
hmm1.smoother()


beta
[[0.1512 0.1616 0.1392]
 [0.4    0.44   0.36  ]
 [1.     1.     1.    ]]
Posterior
[[1.         0.         0.        ]
 [0.05291005 0.23280423 0.71428571]
 [0.04497354 0.24867725 0.70634921]]


array([[1.        , 0.        , 0.        ],
       [0.05291005, 0.23280423, 0.71428571],
       [0.04497354, 0.24867725, 0.70634921]])

In [5]:
# Decoding results using the Viturbi algorithm

hmm1.viturbi()


delta
[[0.6     0.      0.     ]
 [0.012   0.048   0.18   ]
 [0.00384 0.0216  0.0432 ]]
Previous
[[nan nan nan]
 [ 0.  0.  0.]
 [ 1.  2.  2.]]
Path
   Time 1  Time 2  Time 3
0     1.0     3.0     3.0


,Time 1,Time 2,Time 3
0,1.0,3.0,3.0


In [6]:
# One step predictions of hidden and visible states

hmm1.predictor()

Predicted Hidden State
[0.10396825 0.44179894 0.4542328 ]
Predicted Visible State
[0.2415873  0.37671958 0.38169312]


## Investment Decision

In [24]:
## Implement for investment

# Data 
# Transition matrix
TM = np.array([[.6,.3,.1],[.4,.4,.2],[.1,.4,.5]])  
# Observation matrix
OM = np.array([[.45,.4,.15],[.3,.4,.3],[.2,.5,.3]])
OM = OM.T
# Prior probabilities of hidden states
p = [(1/3),(1/3),(1/3)]
# Observed visible states
Viz = [0,2,2]

### a. ### 
The most likely current state given observed states

In [25]:
## The most likely cu

hmm1 = HMM(Viz, TM, OM, p)
hmm1.filter()

Posterior
[[0.45       0.4        0.15      ]
 [0.27258806 0.54364472 0.18376723]
 [0.23871854 0.55697941 0.20430206]]


array([[0.45      , 0.4       , 0.15      ],
       [0.27258806, 0.54364472, 0.18376723],
       [0.23871854, 0.55697941, 0.20430206]])

### b. 

Most likely states at each previous time period

In [26]:
hmm1.smoother()

beta
[[0.0981 0.1142 0.1295]
 [0.3    0.34   0.37  ]
 [1.     1.     1.    ]]
Posterior
[[0.40407323 0.41812357 0.1778032 ]
 [0.24439359 0.55240275 0.20320366]
 [0.23871854 0.55697941 0.20430206]]
alpha
[[0.15       0.13333333 0.05      ]
 [0.02966667 0.05916667 0.02      ]
 [0.00869333 0.02028333 0.00744   ]]


array([[0.40407323, 0.41812357, 0.1778032 ],
       [0.24439359, 0.55240275, 0.20320366],
       [0.23871854, 0.55697941, 0.20430206]])

### c.

Most likely path. NOTE TO GRADER: I did not change this code since it worked for the given code sample. I recognize this is likely wrong.

In [27]:
hmm1.viturbi()

delta
[[0.15       0.13333333 0.05      ]
 [0.018      0.02666667 0.008     ]
 [0.00216    0.00533333 0.0016    ]]
Previous
[[nan nan nan]
 [ 0.  1.  1.]
 [ 0.  1.  1.]]
Path
   Time 1  Time 2  Time 3
0     1.0     1.0     2.0


,Time 1,Time 2,Time 3
0,1.0,1.0,2.0


### d.

NOTE TO GRADER: I attempted to code this but got hung up in the details. Please see my class adjustments for partial credit!

In [28]:
hmm1.predictor()

Posterior
[[0.45       0.4        0.15      ]
 [0.27258806 0.54364472 0.18376723]
 [0.23871854 0.55697941 0.20430206]]


TypeError: 'int' object is not iterable